# Test the `map()` Function to Format Datasets

## Install and Import Dependencies

In [1]:
!pip install -qU datasets pprint

ERROR: Could not find a version that satisfies the requirement pprint (from versions: none)
ERROR: No matching distribution found for pprint


In [ ]:
from datasets import load_dataset
import pprint
import torch

## Load Dataset

In [ ]:
dataset = load_dataset("claudios/code_search_net", "python", split="train[:5]")

In [ ]:
print(dataset)

## Define Prompt Format Function for a Single Sample

In [ ]:
# EOS Token is required to stop open-ended generation and eventual hallucination
# EOS_TOKEN = tokenizer.eos_token
EOS_TOKEN = "<|end_of_text|>"

In [ ]:
def formatFunctionSample(sample):
    language = sample['language']
    instruction = f"What does this {language} function do?"
    inputText = sample['func_code_string']
    outputText = sample['func_documentation_string']

    return {
        "instruction": instruction,
        "input": inputText,
        "output": outputText
    }

## Use the `map()` Method to Format the Dataset

In [ ]:
formattedDataset = dataset.map(formatFunctionSample, remove_columns=dataset.column_names)

## Sample Formatted Dataset

In [ ]:
len(formattedDataset)

In [ ]:
pprint.pp(formattedDataset[1])

## Create Dataset of *Alpaca* Formatted Samples

In [ ]:
sample = formattedDataset[2]

In [ ]:
alpacaFormatString = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

alpacaFormatString += EOS_TOKEN

In [ ]:
print(alpacaFormatString.format(sample['instruction'], sample['input'], sample['output']))

In [ ]:
def formatFunctionDataset(samples):
    alpacaFormattedSamples = []
    for sample in samples:
        alpacaFormattedSamples.append(alpacaFormatString.format(sample['instruction'], sample['input'], sample['output']))
        
    return { "text": alpacaFormattedSamples }

In [ ]:
pprint.pp(formattedDataset[0])

In [ ]:
type(formattedDataset)

In [ ]:
texts = formatFunctionDataset(formattedDataset)
pprint.pp(texts)

In [ ]:
finalDataset = formattedDataset.map(formatFunctionDataset)